In [1]:
import sys
import os

import qgis
from qgis.gui import *
from qgis.core import *
from qgis.utils import plugins
from PyQt5.QtCore import *
from qgis.analysis import QgsNativeAlgorithms

sys.path.append('/usr/share/qgis/python/plugins/')
sys.path.append('/usr/share/qgis/python/')

QgsApplication.setPrefixPath('/usr', True)
app = QgsApplication([], False)
app.initQgis()

import processing
from processing.core.Processing import Processing
Processing.initialize()

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
# for alg in QgsApplication.processingRegistry().algorithms():
#         print(alg.id(), "--->", alg.displayName())

# Stop QGIS appllication
# app.exitQgis()
# app.exit()

True

In [2]:
from qgis.core import QgsProject, QgsApplication
# Get the project instance
project = QgsProject.instance()
# Print the current project file name (might be empty in case no projects have been loaded)
# print(project.fileName())

# Load another project
project.read(f"/home/luca/Code/mappy.git/input_data/qgis_project.qgz")
print(project.fileName())

/home/luca/Code/mappy.git/input_data/qgis_project.qgz


/usr/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
project.mapLayers()
cont = project.mapLayersByName("contacts")[0]
pols = project.mapLayersByName("map")[0]

In [4]:
index = QgsSpatialIndex() # Spatial index

for ft in pols.getFeatures():
    
    index.insertFeature(ft)


<ipython-input-4-22e5363287d2>:5: DeprecationWarning: QgsSpatialIndex.insertFeature() is deprecated
  index.insertFeature(ft)


In [5]:
outtable = []

for f in cont.getFeatures():
    fid = f.id()
    
    g = f.geometry()
    if g.isEmpty() or not f.isValid() or g.length() < 1e-16:
        continue
        
    
    if g.isMultipart():
        for pid, p in enumerate(g.parts()):
            start = p[0]
            end = p[-1]
            
            outtable.append([fid, pid, [start, end]])
        


In [6]:
def find_polygons_containing_point(pt, index, polygons):
    ids = index.intersects(pt.boundingBox()) # polygons intersecting the point
#     request = QgsFeatureRequest()
#     request.setFilterFids(ids)

#     pp = polygons.getFeatures(request)
    pp= polygons.getFeatures(ids)
    real = []
    for p, i in zip(pp, ids):
        if p.geometry().contains(QgsPointXY(pt)):
            real.append(p.id())
            
    return real
            

    

In [7]:
cuttable = []
for fid, pid, (start, end) in outtable:
    mypols = []
    pstart = find_polygons_containing_point(start, index, pols)
    pend = find_polygons_containing_point(end, index, pols)
    mypols += pend + pstart
    
    if len(mypols):
        cuttable.append([fid, pid, mypols])

In [8]:
cuttable

[[32, 0, [22, 29]], [33, 0, [17, 29]], [34, 0, [19, 29]], [35, 0, [11]]]

In [9]:
def get_sel(layer):
    return processing.run("native:saveselectedfeatures", {'INPUT':layer.name(),'OUTPUT':'TEMPORARY_OUTPUT'})["OUTPUT"]

In [10]:
cuttable

[[32, 0, [22, 29]], [33, 0, [17, 29]], [34, 0, [19, 29]], [35, 0, [11]]]

In [11]:
def clip_with_selection(lines, polygons):
    pars = {"INPUT":QgsProcessingFeatureSourceDefinition(lines.name(), True),
            "OVERLAY": QgsProcessingFeatureSourceDefinition(polygons.name(), True),
            "OUTPUT":"memory:"}
    
    
    return processing.run("native:difference", pars)
    
    

In [12]:
# QgsVectorFileWriter.writeAsVectorFormatV2?

In [26]:
for i, (lineid, subid, polygons) in enumerate(cuttable):
    pols.removeSelection()
    cont.removeSelection()
    [pols.select(p) for p in polygons]
    cont.select(lineid)
    
    this_sel_pol = get_sel(pols)
    this_sel_lin = get_sel(cont)
    
    options = QgsVectorFileWriter.SaveVectorOptions()
    options.actionOnExistingFile = QgsVectorFileWriter.CreateOrOverwriteLayer 
    options.layerName = f"pols_{str(i)}"
    options.driverName ="GPKG"
    
    print(QgsVectorFileWriter.writeAsVectorFormatV2(layer=this_sel_pol,fileName="out",transformContext=QgsCoordinateTransformContext(), options=options))
    options.layerName = f"line_{str(i)}"
    print(QgsVectorFileWriter.writeAsVectorFormatV2(layer=this_sel_lin,fileName="out",transformContext=QgsCoordinateTransformContext(), options=options))


    
    o = clip_with_selection(cont, pols)["OUTPUT"]
#     print(len(list(o.getFeatures())))
    
    
    options.layerName = f"clean_line_{str(i)}"
    
    print(QgsVectorFileWriter.writeAsVectorFormatV2(layer=o,fileName="out",transformContext=QgsCoordinateTransformContext(), options=options))
    
    if o.featureCount() > 0:
        f = o.getFeature(1)
        gg = f.geometry()

        cont.startEditing()
        cont.changeGeometry(lineid,gg)
        cont.commitChanges()
#     cont.stopEditing()
#     

QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')    

(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')
(0, '')


<ipython-input-26-d0afb80e317f>:39: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(cont,f"output_pols",'utf-8',driverName='ESRI Shapefile')


(0, '')

In [24]:
cont.commitChanges?

Docstring:
commitChanges(self, stopEditing: bool = True) -> bool
Attempts to commit to the underlying data provider any buffered changes made since the
last to call to :py:func:`~QgsVectorLayer.startEditing`.

Returns the result of the attempt. If a commit fails (i.e. ``False`` is returned), the
in-memory changes are left untouched and are not discarded. This allows editing to
continue if the commit failed on e.g. a disallowed value in a Postgres
database - the user can re-edit and try again.

The commits occur in distinct stages,
(add attributes, add features, change attribute values, change
geometries, delete features, delete attributes)
so if a stage fails, it can be difficult to roll back cleanly.
Therefore any error message returned by :py:func:`~QgsVectorLayer.commitErrors` also includes which stage failed so
that the user has some chance of repairing the damage cleanly.

By setting ``stopEditing`` to ``False``, the layer will stay in editing mode.
Otherwise the layer editing mod

In [49]:
QgsVectorFileWriter.writeAsVectorFormat(cont,"output",'utf-8',driverName='ESRI Shapefile')

<ipython-input-49-edf8a928671d>:1: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(cont,"output",'utf-8',driverName='ESRI Shapefile')


(0, '')

In [50]:
l = cont.getFeature(36)
p = pols.getFeature(30)
a

NameError: name 'a' is not defined

In [ ]:
pols.removeSelection()

In [ ]:
cont.select(36)
pols.select(30)


In [ ]:
pols.selectedFeatureIds()

In [ ]:
print([a.id() for a in QgsApplication.processingRegistry().algorithms() if "clip" in a.id()])

In [ ]:
[2021-02-15 11:39:59] processing.run("native:clip", {'INPUT':QgsProcessingFeatureSourceDefinition('contacts_7768adb1_5e8e_4a87_a7b2_aebd7bb4303e', True),
                                                     'OVERLAY':QgsProcessingFeatureSourceDefinition('map_62aaeba8_b51e_4242_82bc_ea14f8a87fbe', True),'OUTPUT':'TEMPORARY_OUTPUT'})